In [ ]:
import tensorflow as tf

In [ ]:
def neural_net(n_in, n_out, n_hiddens = []):
    """
    Builds a feed forward neural network with `n_in` input
    neurons connected through `len(n_hiddens)` hidden layers,
    each with the amount of neurons specified by the elements
    of `n_hiddens` and leading to an output layer of `n_out` 
    neurons. Returns a list of (weights, biases) tuples which
    can later be fed into the `feed_forward` function.
    """
    network = []
    sizes = [n_in] + n_hiddens + [n_out]
    
    # Loop over all layers in pairs of two adjacent at a time.
    for left, right in zip(sizes[:-1], sizes[1:]):
        weights = tf.Variable(tf.truncated_normal([left, right]))
        biases = tf.Variable(tf.zeros([right]))
        network.append((weights, biases))
        
    return network

In [ ]:
def feed_forward(layers, data):
    """
    Applies the feed forward neural network given in
    layers to the data.
    """
    for (weights, biases) in layers[:-1]: # All but last.
        data = tf.sigmoid(tf.matmul(data, weights) + biases)
    
    # Output neurons do not use an activation function.
    weights_out, biases_out = layers[-1]
    return tf.matmul(data, weights_out) + biases_out

In [ ]:
def accuracy(y, y_):
    """
    Calculates the accuracy of the predictions given in y
    in relation to the actual target labels given in y_.
    """
    correct_predictions = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    return tf.reduce_mean(tf.cast(correct_predictions, dtype=tf.float32))

In [ ]:
from random import shuffle

def ffnn(train, test = None, epochs=10000, rate=0.01, hidden=[]):
    data, targets = train
    
    samples, n_in  = data.shape
    _,       n_out = targets.shape

    # build up TensorFlow data flow graph by initializing variables and
    # placeholders and composing functions.
    x = tf.placeholder(tf.float32, shape=[None, n_in])  # Input
    y = tf.placeholder(tf.float32, shape=[None, n_out]) # Target Output
    
    model = neural_net(n_in, n_out, n_hiddens=hidden) # Network.
    pred  = feed_forward(model, x) # Network Output.
    
    # Cost function and cost optimizer/training step.
    cost_fn = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
    optimizer = tf.train.GradientDescentOptimizer(rate).minimize(cost_fn)
    
    # If no test data is provided we test the network using the training data.
    test_data, test_targets = test if test is not None else train
    test_pred = tf.nn.softmax(feed_forward(model, test_data))
    test_accu = accuracy(test_pred, y)

    # Range used for sampling inside the training loop.
    take_all = take = [i for i in range(n_in)]
    
    with tf.Session() as sess:
        init = tf.initialize_all_variables()
        sess.run(init)

        # Training loop.
        for epoch in range(1, epochs+1):
            # If there is a lot of test data take a new 
            # random sample in each iteration.
            if n_in > 500:
                shuffle(take_all)
                take = take_all[0:500]
            
            # Run the optimizer with a sample of the data.
            sess.run(optimizer, feed_dict={ x: data[take], y: targets[take] })
            
            # Print the progress and accuracy when using the test data.
            if (epoch % 200) == 0:
                performance = sess.run(test_accu, feed_dict={y: test_targets})
                print('Epoch %d with accuracy %.2f%%.' % (epoch, performance*100))

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
data, targets = mnist.train.next_batch(55000)
ffnn(train=(data, targets), test=(mnist.test.images, mnist.test.labels), hidden=[200], epochs=20000)

## MNIST Results
# hidden=[200], epochs=20000, GradiendDescentOptimizer, softmax_cross_entropy_with_logits --> 71.83%
# hidden=[128, 32], epochs=20000, GradiendDescentOptimizer, softmax_cross_entropy_with_logits --> 73.29%